# Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

# Gather Data

In [2]:
twitter = pd.read_json(r"C:\Programming\Python\Programs_NLP\MLFinalProject\src\data\twitter_data_with_sentiment.json")

In [3]:
twitter.timestamp = pd.to_datetime(twitter.timestamp)

In [4]:
stocks = pd.read_csv(r"C:\Programming\Python\Programs_NLP\MLFinalProject\src\data\stock_doj_mention_only.csv")

In [5]:
stocks.date = pd.to_datetime(stocks.date)

In [6]:
stocks = stocks.set_index(stocks.date)

In [7]:
stock_symbols = set(stocks.columns[1:])

In [8]:
twitter_symbols = set(twitter.symbols.apply(str.lower))

In [14]:
new_df_cols = ['symbols','sentiment_compound', 'sentiment_neg', 'sentiment_pos','timestamp',
               'sentiment_neu', 'id', 't-3', 't-2', 't-1', 't0', 't1', 't2', 't3']

time_lags = ['t-3', 't-2', 't-1', 't0', 't1', 't2', 't3']

new_df_data = {col:[] for col in new_df_cols}

for tup in tqdm_notebook(twitter.itertuples(), total=twitter.shape[0]):
    symbol = tup.symbols.lower()
    
    # Skip symbols we don't have data for
    if symbol not in stock_symbols:
        continue

    # Check if the tweet happens before, during, or after the day
    
    # -1 if before open
    # 0 if during trading
    # 1 if after trading
    occurance = 0
    
    if tup.timestamp.hour + tup.timestamp.minute/60 < 9.5:
        occurance = -1
    elif tup.timestamp.hour > 16:
        occurance = 1
    
    # Get slice of stock DF
    stock_data = stocks[[symbol]].loc[tup.timestamp - pd.Timedelta(days=3):tup.timestamp + pd.Timedelta(days=3)].copy()
    stock_data['lag_day'] = None
    
#     if not stock_data[symbol].isnull().all():
#         print("YAY")
    
    date = tup.timestamp.date()
    
    # If tweet happened before open, then t0 is the day of the tweet,
    # otherwise it's the day after
    if occurance < 0:
        before_range = list(range(0-(stock_data.loc[:date].shape[0]-1),0))
        after_range = list(range(0,stock_data[date:].shape[0]))
        
    else:
        before_range = list(range(0-(stock_data.loc[:date + pd.Timedelta(days=1)].shape[0]-1),0))
        after_range = list(range(0,stock_data[date + pd.Timedelta(days=1):].shape[0]))
    
    try:
        stock_data['lag_day'] = before_range + after_range
    except ValueError:
        print('Error...skipping')
        continue
    
    lags = []
    for _,close,lag in stock_data.itertuples():
        lag_col = 't' + str(lag)
        lags.append(lag_col)
        new_df_data[lag_col].append(close)
    
    for col in time_lags:
        if col not in lags:
            new_df_data[col].append(np.nan)
            
    for col in new_df_cols:
        if getattr(tup,col, None) is not None:
            new_df_data[col].append(getattr(tup,col))


Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skipping
Error...skippi

In [15]:
df = pd.DataFrame.from_dict(new_df_data)

In [16]:
df[~(df.t0.isna())]

,symbols,sentiment_compound,sentiment_neg,sentiment_pos,timestamp,sentiment_neu,id,t-3,t-2,t-1,t0,t1,t2,t3
532,ABC,0.0000,0.000,0.000,2018-07-17 01:59:34,1.000,1019038873572335600,NaN,NaN,86.90,86.82,86.33,85.56,84.7
954,ABC,0.2960,0.000,0.180,2018-07-17 13:49:01,0.820,1019217411524972500,NaN,86.90,86.82,86.33,85.56,84.70,NaN
959,ABC,0.0000,0.000,0.000,2018-07-17 13:49:22,1.000,1019217499861274600,NaN,86.90,86.82,86.33,85.56,84.70,NaN
979,ABC,-0.2732,0.139,0.000,2018-07-17 14:18:38,0.861,1019224865117802500,NaN,86.90,86.82,86.33,85.56,84.70,NaN
1744,ADBE,0.0000,0.000,0.000,2018-07-18 01:26:09,1.000,1019392850776035300,NaN,254.68,258.31,259.78,257.68,257.54,NaN
1784,ADBE,0.3612,0.000,0.161,2018-07-18 02:01:13,0.839,1019401673649901600,NaN,254.68,258.31,259.78,257.68,257.54,NaN
1799,ADBE,0.0000,0.000,0.000,2018-07-18 02:16:27,1.000,1019405509626138600,NaN,254.68,258.31,259.78,257.68,257.54,NaN
1822,ADBE,0.0000,0.000,0.000,2018-07-18 02:39:54,1.000,1019411409371844600,NaN,254.68,258.31,259.78,257.68,257.54,NaN
2353,ADBE,0.0000,0.000,0.000,2018-07-18 11:11:52,1.000,1019540249179930600,254.68,258.31,259.78,257.68,257.54,NaN,NaN
2381,ADBE,0.0000,0.000,0.000,2018-07-18 11:30:06,1.000,1019544838377787400,254.68,258.31,259.78,257.68,257.54,NaN,NaN
